In [66]:
from dateutil.parser import parse
from datetime import datetime
import numpy as np
import pandas as pd
import time

In [67]:
data = pd.read_csv('/Users/liuyang/Desktop/cleaned_2018_08.csv')

In [70]:
def minutes_trans(newData):
  newData['tpep_pickup_datetime'] = newData['tpep_pickup_datetime'].map(lambda x: parse(x).strftime('%Y/%m/%d %H:%M'))
  newData['tpep_pickup_datetime'] = newData['tpep_pickup_datetime'].map(lambda x: datetime.strptime(x,'%Y/%m/%d %H:%M'))
  return newData

In [71]:
newData = minutes_trans(data)

In [73]:
# generate time series
def generate_countinous(start ='6/01/2018', end ='7/1/2018'):
  test_date = pd.date_range(start,end, freq='T')
  test_date = pd.DataFrame(test_date)
  test_date.rename(columns = {0:'pickup_time_g'}, inplace = True)
  test_date.drop([len(test_date)-1], inplace=True)
  return test_date

In [8]:
test_date = generate_countinous('8/01/2018','9/1/2018')

In [74]:
#extract the data slice that are wanted
#### really important stage for extraction ---- later
def data_extract(data, number):
  data_index = data[data['PULocationID'] == number].index.to_list()
  newData = data.iloc[data_index,:]
  return newData

In [0]:
cleaned_data = data_extract(newData, 237)

In [75]:
### try
def minutes_accumulate(cleaned_data):
  Counted_value = cleaned_data.groupby('tpep_pickup_datetime', as_index=False).size()
  avg_price = cleaned_data.groupby('tpep_pickup_datetime')['total_amount'].mean()
  avg_duration = cleaned_data.groupby('tpep_pickup_datetime')['duration'].mean()
  new = pd.concat([Counted_value,avg_price,avg_duration], axis=1)
  new.reset_index(inplace=True,drop=False)
  return new
#all(Counted_value.index == avg_price.index)

In [0]:
# merge the result and macth in minutes
cleaned_data = minutes_accumulate(cleaned_data)
matched_date = pd.merge(test_date, cleaned_data, left_on = 'pickup_time_g', right_on = 'tpep_pickup_datetime', how = 'left')

In [76]:
# if there is no order at this moment, we set this value to 999
#remove and fill in null values
def null_value_implaint(matched_date):
  del matched_date['tpep_pickup_datetime']
  matched_date['total_amount'] = matched_date['total_amount'].fillna(0)
  matched_date['duration'] = matched_date['duration'].fillna(999)
  matched_date[0] = matched_date[0].fillna(0)
  return matched_date

In [0]:
matched_date = null_value_implaint(matched_date)

In [77]:
def interval_extraction(dataset, window_num):
    Demand = pd.Series(matched_date[0]).rolling(window = window_num).sum()
    Avg_Price = pd.Series(matched_date['total_amount']).rolling(window = window_num).mean()
    Avg_Duration = pd.Series(matched_date['duration']).rolling(window = window_num).mean()
    #check the number of null values
    #tryit.isna().sum()
    Demand.dropna(inplace=True)
    Avg_Price.dropna(inplace=True)
    Avg_Duration.dropna(inplace=True)
    Demand.reset_index(drop=True,inplace = True)
    Avg_Price.reset_index(drop=True,inplace = True)
    Avg_Duration.reset_index(drop=True,inplace = True)
    cleaned_data = pd.concat([Demand,Avg_Price,Avg_Duration], axis=1)
    cleaned_data.rename(columns={0:'Demand_15min','total_amount':'Avg_profit', 'duration':'Avg_duration'}, inplace=True)
    new = pd.concat([dataset,cleaned_data], axis=1)
    new.dropna(inplace=True)
    return new

In [0]:
new = interval_extraction(matched_date, 15)

In [78]:
def separate_timestamp(matched_date):
    matched_date['weekday'] = matched_date['pickup_time_g'].dt.weekday
    matched_date['month'] = matched_date['pickup_time_g'].map(lambda x: x.month)
    matched_date['hour'] = matched_date['pickup_time_g'].map(lambda x: x.hour)
    matched_date['minute'] = matched_date['pickup_time_g'].map(lambda x: x.minute)
    matched_date['second'] = matched_date['pickup_time_g'].map(lambda x: x.second)
    matched_date['day'] = matched_date['pickup_time_g'].map(lambda x: x.day)
    return matched_date

In [0]:
matched_date = separate_timestamp(new)

In [79]:
def num_attach(dataset,number):
  locationID = [number] * len(matched_date)
  matched_date['locationID'] = pd.Series(locationID)
  return matched_date

In [80]:
a = list(newData['DOLocationID'].unique())

In [84]:
# write directly 
start = time.time()
for number in a:
    cleaned_data = data_extract(newData, number)
    cleaned_data = minutes_accumulate(cleaned_data)
    matched_date = pd.merge(test_date, cleaned_data, left_on = 'pickup_time_g', right_on = 'tpep_pickup_datetime', how = 'left')
    matched_date = null_value_implaint(matched_date)
    new = interval_extraction(matched_date, 15)
    matched_date = separate_timestamp(new)
    key1 = num_attach(matched_date, number)
    key1.to_csv(f'/Users/liuyang/Desktop/2018_08/{number}.csv')
end = time.time()
print(end-start)